## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,39.20,93,90,32.21,snow
1,1,Parintins,BR,-2.6283,-56.7358,87.03,61,89,4.03,heavy intensity rain
2,2,Punta Arenas,CL,-53.1500,-70.9167,48.20,57,40,20.71,scattered clouds
3,3,Tahe,CN,52.3333,124.7333,19.81,91,75,3.62,broken clouds
4,4,Bengkulu,ID,-3.8004,102.2655,76.44,88,96,13.13,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
user_min = float(input("What is the minimum temperature you would like for your trip? "))
user_max = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_df = city_data_df.loc[(city_data_df["Max Temp"] >= user_min) & (city_data_df["Max Temp"] <= user_max)]

In [15]:
# 4a. Determine if there are any empty rows.
filtered_df.count()

City_ID                236
City                   236
Country                234
Lat                    236
Lng                    236
Max Temp               236
Humidity               236
Cloudiness             236
Wind Speed             236
Current Description    236
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = filtered_df.dropna()

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Parintins,BR,87.03,heavy intensity rain,-2.6283,-56.7358,
4,Bengkulu,ID,76.44,overcast clouds,-3.8004,102.2655,
8,Obo,CF,82.62,overcast clouds,5.3998,26.4869,
9,Rikitea,PF,77.18,broken clouds,-23.1203,-134.9692,
10,Caravelas,BR,82.15,broken clouds,-17.7125,-39.2481,
14,Humaita,BR,89.64,light rain,-7.5061,-63.0208,
15,Coahuayana,MX,84.20,few clouds,18.7333,-103.6833,
16,Anito,PH,77.65,few clouds,12.4497,125.2886,
19,Mana,GF,81.88,overcast clouds,5.6592,-53.7767,
22,Saint-Philippe,RE,80.60,light rain,-21.3585,55.7679,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params["location"] = f"{lat},{lng}"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"{row['City']} - Hotel not found... skipping")

Obo - Hotel not found... skipping
Goure - Hotel not found... skipping
Wattegama - Hotel not found... skipping
Manuk Mangkaw - Hotel not found... skipping
Tanout - Hotel not found... skipping
Chunhuhub - Hotel not found... skipping
Amapa - Hotel not found... skipping
Qandala - Hotel not found... skipping
Bozoum - Hotel not found... skipping
Micheweni - Hotel not found... skipping
Thinadhoo - Hotel not found... skipping


In [26]:
# 7. Drop the rows where there is no Hotel Name.
# Save the hotel DataFrame rows to itself only if there is a Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Parintins,BR,87.03,heavy intensity rain,-2.6283,-56.7358,Cama e Cafe
4,Bengkulu,ID,76.44,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
9,Rikitea,PF,77.18,broken clouds,-23.1203,-134.9692,Pension Maro'i
10,Caravelas,BR,82.15,broken clouds,-17.7125,-39.2481,Pousada dos Navegantes
14,Humaita,BR,89.64,light rain,-7.5061,-63.0208,Corrêa Hotel
15,Coahuayana,MX,84.20,few clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
16,Anito,PH,77.65,few clouds,12.4497,125.2886,The Yellow House
19,Mana,GF,81.88,overcast clouds,5.6592,-53.7767,LE SAMANA
22,Saint-Philippe,RE,80.60,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
26,Avarua,CK,78.80,scattered clouds,-21.2078,-159.7750,Paradise Inn


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))